In [ ]:
from imports import *
plt.rc('figure', max_open_warning=200)
%matplotlib notebook
sns.set_theme()

# Loading  preprosessed data

In [ ]:
# Initialize path variables for main folders

print(os.getcwd())

# Path for saving epoch data and features
ft_dir_path = os.path.join(os.getcwd(), 'features')
data_dir_path = os.path.join(os.getcwd(), 'word_data')


In [ ]:
raw = mne.io.read_raw_fif(data_dir_path + 'reref_filterd_raw.fif', preload=True)


# Epoching data

In [ ]:
gc.collect()

## Global variables & filtering

In [ ]:
# Frequency bands

bands = [(0.9, 4, 'Delta (0.9-4 Hz)', 'D'), (4, 8, 'Theta (4-8 Hz)', 'T'), (8, 14, 'Alpha (8-14 Hz)', 'A'), 
         (14, 25, 'Beta (14-25 Hz)', 'B'), (25, 40, 'Gamma (25-40 Hz)', 'G')]

str_freq_rr = [bands[i][3] for i in range(len(bands))] #bands names 'D', 'T', 'A', 'B', 'G'
n_freq = len(str_freq_rr) # 5

In [ ]:
# Localization by scalp regions

regions = [(['Fp1','Fp2','Fpz'], 'Fp', 'Pre-frontal'),
           (['AF7, AF3','AF4','AF8'], 'AF', 'In-between frontal'),
           (['F9','F7','F5','F3','FT9','FT7','FC5','FC3'], 'LF', 'Left Frontal'),
           (['F1','Fz','F2','FC1','FCz','FC2'], 'MF', 'Midline Frontal'),
           (['F4','F6','F8','F10','FC4','FC6','FT8','FT10'], 'RF', 'Right Frontal'),
           (['T7','TP9','TP7'], 'LT', 'Left Temporal'),
           (['T8','TP8','TP10'], 'RT', 'Right Temporal'),
           (['C5','C3','CP5','CP3'], 'LC', 'Left Central'),
           (['C1','Cz','C2','CP1','CPz', 'CP2'], 'MC', 'Midline Central'),
           (['C4','C6','CP4','CP6'], 'RC', 'Right Central'),
           (['P9','P7','P5','P3'], 'LP', 'Left Parietal'),
           (['P1','Pz','P2'], 'MP', 'Midline Parietal'),
           (['P4','P6','P8','P10'], 'RP', 'Right Parietal'),
           (['PO9','PO7','PO3','O1'], 'LO', 'Left Occipital'),
           (['POz','Oz'], 'MO', 'Midline Occipital'),
           (['PO4','PO8','PO10','O2'], 'RO', 'Right Occipital')]


n_regions = len(regions)


## Epoching

In [ ]:
gc.collect()

In [ ]:
# Epochs by fixed length events, duration 5s, overlap 0.2s (re-referenced)

sec5_events = mne.make_fixed_length_events(raw, start=0.5, duration=5.)
kwargs = dict(baseline=None, tmin=-0.5, tmax=0.5, preload=True)
sec5_epochs = mne.Epochs(raw, sec5_events.astype(int), **kwargs)[2:-2]



In [ ]:
sec5_epochs.average().plot_joint()
print()

In [ ]:
# Global variables

ch_names = sec5_epochs.ch_names
n_freq = len(str_freq_rr)
n_channels = len(ch_names)

n_samples = sec5_epochs.__len__()
n_times = len(sec5_epochs.get_data()[0,0,:])

sampling_rate = raw.info['sfreq']

# Extract features

## PSD features

In [ ]:
# Calculating PSD for re-referenced epochs (Multitaper)

kwargs = dict(fmin=bands[0][0], fmax=bands[-1][1], sfreq=sampling_rate, bandwidth=None, adaptive=True, n_jobs=1)
rr_psd_mtaper, rr_freq_mtaper = psd_array_multitaper(sec5_epochs.get_data(), **kwargs)



In [ ]:
# print(rr_freq_mtaper) # 0.9 - 40
# print(rr_psd_mtaper.shape) # (2497, 70, 40)

In [ ]:
freq_masks = [(fmin < rr_freq_mtaper) & (rr_freq_mtaper < fmax) for (fmin, fmax, _, _) in bands]
loc_masks = [[ch_names[i] in reg for i in range(n_channels)] for (reg, _, _) in regions] # 16 x 70
# loc_plt_masks = [[ch_names[i] in reg for i in range(n_channels)] for (reg, _, _) in regions_plt] # 9 x 70


In [ ]:
# for each frequency band (2497, 70, 5)
ft_psd_spectr_raw = np.array([np.mean(rr_psd_mtaper[:,:, _freq_mask], axis=2) for _freq_mask in freq_masks]).transpose(1,2,0)
# for each freq. band + for region (2497, 16, 5)
ft_psd_sp_loc_raw = np.array([np.mean(ft_psd_spectr_raw[:,_mask,:], axis=1) for _mask in loc_masks]).transpose(1,0,2)
# average among channels by epoch for each band (2497, 5)
ft_psd_sp_all_raw = np.mean(ft_psd_spectr_raw, axis=1)


In [ ]:
ft_psd_spectr_db = 10 * np.log10(ft_psd_spectr_raw) # Convert psd to dB format
ft_psd_sp_loc_db = 10 * np.log10(ft_psd_sp_loc_raw) # Convert psd to dB 
ft_psd_sp_all_db = 10 * np.log10(ft_psd_sp_all_raw) # Convert psd to dB format

In [ ]:

df_ft_psd_raw = pd.DataFrame()
df_ft_psd_db = pd.DataFrame()

df_ft_psd_loc_raw = pd.DataFrame()
df_ft_psd_loc_db = pd.DataFrame()

df_ft_psd_all_raw = pd.DataFrame()
df_ft_psd_all_db = pd.DataFrame()

In [ ]:
for i in range(n_freq): # 5
    for j in range(n_channels): # 70
        df_ft_psd_raw[str_freq_rr[i]+'_psd_'+ch_names[j]] = ft_psd_spectr_raw[:,j,i]
        df_ft_psd_db[str_freq_rr[i]+'_psd_'+ch_names[j]] = ft_psd_spectr_db[:,j,i]
    for j in range(n_regions):    
        df_ft_psd_loc_raw[str_freq_rr[i]+'_psd_'+regions[j][1]] = ft_psd_sp_loc_raw[:,j,i]
        df_ft_psd_loc_db[str_freq_rr[i]+'_psd_'+regions[j][1]] = ft_psd_sp_loc_db[:,j,i]
    df_ft_psd_all_raw[str_freq_rr[i]+'_psd_All'] = ft_psd_sp_all_raw[:,i]
    df_ft_psd_all_db[str_freq_rr[i]+'_psd_All'] = ft_psd_sp_all_db[:,i]



In [ ]:
# Scaling dB re-referenced data
ft_psd_db_sc = StandardScaler().fit_transform(df_ft_psd_db.to_numpy())
df_ft_psd_db_sc = pd.DataFrame(ft_psd_db_sc, columns=df_ft_psd_db.columns)

ft_psd_loc_db_sc = StandardScaler().fit_transform(df_ft_psd_loc_db.to_numpy())
df_ft_psd_loc_db_sc = pd.DataFrame(ft_psd_loc_db_sc, columns=df_ft_psd_loc_db.columns)

ft_psd_all_db_sc = StandardScaler().fit_transform(df_ft_psd_all_db.to_numpy())
df_ft_psd_all_db_sc = pd.DataFrame(ft_psd_all_db_sc, columns=df_ft_psd_all_db.columns)

## PSD Indices

In [ ]:
df_ft_psd_ind = pd.DataFrame()

# theta/delta, alpha/delta, alpha/theta, alpha/(delta+theta), 
# beta/delta, beta/theta, beta/alpha, beta/(delta+theta), 
# beta/(theta+alpha), gamma/delta, gamma/theta, gamma/alpha, gamma/beta, 
# gamma/(delta+theta), gamma/(theta+alpha) and gamma /(alpha+beta)
str_psd_ind = ['T_D','A_D','A_T','A_DT','B_D','B_T','B_A','B_DT','B_TA','G_D','G_T','G_A','G_B','G_DT','G_TA','G_AB']

df_ft_psd_ind_loc = pd.DataFrame()
df_ft_psd_ind_all = pd.DataFrame()

# Indices per region (averaged PSD)
for _r in range(n_regions):
    for ind in str_psd_ind:
        if (len(ind)==3):
            df_ft_psd_ind_loc[ind+'_psd_'+regions[_r][1]] = (df_ft_psd_loc_raw[ind[0]+'_psd_'+regions[_r][1]] / 
                                                             df_ft_psd_loc_raw[ind[2]+'_psd_'+regions[_r][1]])
        elif (len(ind)==4):
            df_ft_psd_ind_loc[ind+'_psd_'+regions[_r][1]] = (df_ft_psd_loc_raw[ind[0]+'_psd_'+regions[_r][1]] / 
                                                            (df_ft_psd_loc_raw[ind[2]+'_psd_'+regions[_r][1]]+
                                                             df_ft_psd_loc_raw[ind[3]+'_psd_'+regions[_r][1]]))

# Indices for all channels averaged PSD
for ind in str_psd_ind:
    if (len(ind)==3):
        df_ft_psd_ind_all[ind+'_psd_All'] = (df_ft_psd_all_raw[ind[0]+'_psd_All'] / 
                                             df_ft_psd_all_raw[ind[2]+'_psd_All'])
    elif (len(ind)==4):
        df_ft_psd_ind_all[ind+'_psd_All'] = (df_ft_psd_all_raw[ind[0]+'_psd_All'] / 
                                            (df_ft_psd_all_raw[ind[2]+'_psd_All']+
                                             df_ft_psd_all_raw[ind[3]+'_psd_All']))


In [ ]:
# Log-scaling PSD indices (dB format)
df_ft_psd_ind_loc_log = 10 * np.log10(df_ft_psd_ind_loc)
df_ft_psd_ind_all_log = 10 * np.log10(df_ft_psd_ind_all)

# Scaling
ft_psd_ind_loc_sc = StandardScaler().fit_transform(df_ft_psd_ind_loc_log.to_numpy())
df_ft_psd_ind_loc_sc = pd.DataFrame(ft_psd_ind_loc_sc, columns=df_ft_psd_ind_loc_log.columns)

ft_psd_ind_all_sc = StandardScaler().fit_transform(df_ft_psd_ind_all_log.to_numpy())
df_ft_psd_ind_all_sc = pd.DataFrame(ft_psd_ind_all_sc, columns=df_ft_psd_ind_all_log.columns)


## Coherence & PLV features

In [ ]:
# Averaging epochs by region
loc_masks = [[ch_names[i] in reg for i in range(n_channels)] for (reg, _, _) in regions]

# Re-referenced data
ft_epochs = sec5_epochs.get_data()
ft_epochs_loc = np.array([np.mean(ft_epochs[:,_mask,:], axis=1) for _mask in loc_masks]).transpose(1,0,2)

print(ft_epochs.shape)
print(ft_epochs_loc.shape)


In [ ]:
# Calculating CSD (Cross-spectral densities), re-referenced data

ft_csd_matr_sp = []
ft_csd_matr_loc_sp = []
kwargs = dict(fmin=bands[0][0], fmax=bands[-1][1], sfreq=sampling_rate, adaptive=True, n_jobs=-1, verbose='DEBUG')

# Calculating CSD for each epoch (Multitaper)
for i in range(n_samples):
    csd_mtaper = csd_array_multitaper(ft_epochs[i].reshape((1, n_channels, n_times)), **kwargs)
    ft_csd_matr_sp.append([csd_mtaper.mean(fmin, fmax).get_data() for (fmin, fmax, _, _) in bands])

    csd_mtaper = csd_array_multitaper(ft_epochs_loc[i].reshape((1, n_regions, n_times)), **kwargs)
    ft_csd_matr_loc_sp.append([csd_mtaper.mean(fmin, fmax).get_data() for (fmin, fmax, _, _) in bands])
    print(i)

ft_csd_matr_sp = np.array(ft_csd_matr_sp)
ft_csd_matr_loc_sp = np.array(ft_csd_matr_loc_sp)

print(ft_csd_matr_sp.shape)
print(ft_csd_matr_loc_sp.shape)

#### save calculated csd values

In [ ]:
# np.save('csd.npy', ft_csd_matr_sp)
# np.save('csd_loc.npy', ft_csd_matr_loc_sp)

##### download calculated csd values

In [ ]:
# ft_csd_matr_sp = np.load('csd.npy')
# ft_csd_matr_loc_sp = np.load('csd_loc.npy')

In [ ]:
# Calculating Coherence, PLV and PSD from CSD, re-referenced data
SLICE_LEN = 3

df_ft_coh = pd.DataFrame()
df_ft_plv = pd.DataFrame()
df_ft_coh_loc = pd.DataFrame()
df_ft_plv_loc = pd.DataFrame()

for _freq in range(n_freq): # frequency band
    # By channel pairs
    for i in range(n_channels): # channel
        for j in range(i+1, n_channels): #channel
            coh_list = []
            plv_list = []
            for _samp in range(n_samples): #epoch
                samp_slice = ft_csd_matr_sp[max(_samp-SLICE_LEN//2, 0):min(_samp+SLICE_LEN//2+SLICE_LEN%2, n_samples), _freq,:,:]
                coh = np.abs(np.mean(samp_slice[:,i,j])) / sqrt(np.mean(samp_slice[:,i,i]).real * np.mean(samp_slice[:,j,j]).real)
                plv = np.abs(np.mean(samp_slice[:,i,j]/np.abs(samp_slice[:,i,j])))
                
                coh_list.append(coh)
                plv_list.append(plv)
                
            df_ft_coh[str_freq_rr[_freq]+'_coh_'+ch_names[i]+'_'+ch_names[j]] = np.array(coh_list)
            df_ft_plv[str_freq_rr[_freq]+'_plv_'+ch_names[i]+'_'+ch_names[j]] = np.array(plv_list)

    # By region pairs
    for i in range(n_regions): # region mean
        for j in range(i+1, n_regions): # region mean
            coh_list = []
            plv_list = []
            for _samp in range(n_samples): # epoch
                samp_slice = ft_csd_matr_loc_sp[_samp:min(_samp+SLICE_LEN, n_samples),_freq,:,:]
                coh = np.abs(np.mean(samp_slice[:,i,j])) / sqrt(np.mean(samp_slice[:,i,i]).real * np.mean(samp_slice[:,j,j]).real)
                plv = np.abs(np.mean(samp_slice[:,i,j]/np.abs(samp_slice[:,i,j])))
                
                coh_list.append(coh)
                plv_list.append(plv)
            df_ft_coh_loc[str_freq_rr[_freq]+'_coh_'+regions[i][1]+'_'+regions[j][1]] = np.array(coh_list)
            df_ft_plv_loc[str_freq_rr[_freq]+'_plv_'+regions[i][1]+'_'+regions[j][1]] = np.array(plv_list)   
    #print(len(df_ft_plv_loc.columns))
    

In [ ]:
# Special coherence & PLV features

df_ft_coh_ind = pd.DataFrame()
df_ft_plv_ind = pd.DataFrame()
df_ft_coh_ind_loc = pd.DataFrame()
df_ft_plv_ind_loc = pd.DataFrame()
df_ft_coh_ind_all = pd.DataFrame()
df_ft_plv_ind_all = pd.DataFrame()

for _freq in range(n_freq):  # frequency band
    # By channel pairs
    for _ch in range(n_channels):  # channel
        # Coherence
        ch_cols = [col for col in df_ft_coh.columns if col.startswith(str_freq_rr[_freq]) and (ch_names[_ch] in col)]
        for threshold in [0.6, 0.7, 0.8]:
            df_ft_coh_ind[str_freq_rr[_freq] + '_coh_' + ch_names[_ch] + f'_0{threshold * 10}'] = (
                    df_ft_coh[ch_cols] >= threshold).sum(axis=1)
            
        # PLV
        ch_cols = [col for col in df_ft_plv.columns if col.startswith(str_freq_rr[_freq]) and (ch_names[_ch] in col)]
        for threshold in [0.6, 0.7, 0.8]:
            df_ft_plv_ind[str_freq_rr[_freq] + '_plv_' + ch_names[_ch] + f'_0{threshold * 10}'] = (
                    df_ft_plv[ch_cols] >= threshold).sum(axis=1)

#     By region
    for _reg in range(n_regions):
        for threshold_delim in ['06', '07', '08']:
            # Coherence
            reg_cols = [col for col in df_ft_coh_ind.columns if any(ch in col for ch in regions[_reg][0]) and
                        col.startswith(str_freq_rr[_freq]) and (threshold_delim in col)]
            df_ft_coh_ind_loc[str_freq_rr[_freq] + '_coh_' + regions[_reg][1] + '_'+ threshold_delim] = df_ft_coh_ind[
                reg_cols].mean(axis=1)

            # PLV
            reg_cols = [col for col in df_ft_plv_ind.columns if any(ch in col for ch in regions[_reg][0]) and
                        col.startswith(str_freq_rr[_freq]) and (threshold_delim in col)]
            df_ft_plv_ind_loc[str_freq_rr[_freq] + '_plv_' + regions[_reg][1] + '_' + threshold_delim] = df_ft_plv_ind[
                reg_cols].mean(axis=1)

    # Averaged by all channels
    for threshold_delim in ['06', '07', '08']:
        # Coherence
        reg_cols = [col for col in df_ft_coh_ind.columns if
                    col.startswith(str_freq_rr[_freq]) and (threshold_delim in col)]
        df_ft_coh_ind_all[str_freq_rr[_freq] + '_coh_all_' + threshold_delim] = df_ft_coh_ind[reg_cols].mean(axis=1)
        
        # PLV
        reg_cols = [col for col in df_ft_plv_ind.columns if
                    col.startswith(str_freq_rr[_freq]) and (threshold_delim in col)]
        df_ft_plv_ind_all[str_freq_rr[_freq] + '_plv_all_' + threshold_delim] = df_ft_plv_ind[reg_cols].mean(axis=1)


display(df_ft_coh_ind_all)


# Saving data

In [ ]:
# Saving epochs

sec5_epochs.save(os.path.join(ft_dir_path, 'epochs.fif'), overwrite=True)
print(sec5_epochs.get_data().shape)


In [ ]:
# Saving main features DataFrames

print(ft_dir_path)
ft_dir_path = '/home/lipperrdino/verenv/features'
df_ft_psd_loc_db.to_feather(os.path.join(ft_dir_path, 'df_ft_psd_loc_db.feather'))
df_ft_psd_all_db.to_feather(os.path.join(ft_dir_path, 'df_ft_psd_all_db.feather'))
df_ft_psd_ind_loc_log.to_feather(os.path.join(ft_dir_path, 'df_ft_psd_ind_loc_log.feather'))
df_ft_psd_ind_all_log.to_feather(os.path.join(ft_dir_path, 'df_ft_psd_ind_all_log.feather'))

df_ft_coh.to_feather(os.path.join(ft_dir_path, 'df_ft_coh.feather'))
df_ft_plv.to_feather(os.path.join(ft_dir_path, 'df_ft_plv.feather'))
df_ft_coh_loc.to_feather(os.path.join(ft_dir_path, 'df_ft_coh_loc.feather'))
df_ft_plv_loc.to_feather(os.path.join(ft_dir_path, 'df_ft_plv_loc.feather'))

df_ft_coh_ind_loc.to_feather(os.path.join(ft_dir_path, 'df_ft_coh_ind_loc.feather'))
df_ft_plv_ind_loc.to_feather(os.path.join(ft_dir_path, 'df_ft_plv_ind_loc.feather'))
df_ft_coh_ind_all.to_feather(os.path.join(ft_dir_path, 'df_ft_coh_ind_all.feather'))
df_ft_plv_ind_all.to_feather(os.path.join(ft_dir_path, 'df_ft_plv_ind_all.feather'))